In [ ]:
from PIL import Image, ImageFilter, ImageChops
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
NUM_LAYER = 10
ITERATIONS = 20
STEP = 0.1
OCTAVE_SCALE = 1.4
NUM_OCTAVE = 3

LOAD_PATH = "nsfw.pth"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

deprocess = lambda x: x * torch.Tensor([0.229, 0.224, 0.225]).to(DEVICE) + \
        torch.Tensor([0.485, 0.456, 0.406]).to(DEVICE)

In [ ]:
model = torchvision.models.inception_v3()
model.fc = nn.Linear(model.fc.in_features, 5)
model.load_state_dict(torch.load(LOAD_PATH))
model = model.to(DEVICE)
# model.eval()
modules_list = list(model.children())

In [ ]:
def helper(image, num_layer, iterations, step):        
    x = preprocess(image).unsqueeze(0).to(DEVICE)
    x = torch.autograd.Variable(x, requires_grad=True)

    model.zero_grad()

    for i in range(iterations):
        y = x

        for i in range(num_layer):
            y = modules_list[i](y)

        loss = y.norm()
        loss.backward()

        x.data = x.data + step * x.grad.data

    x = x.data.squeeze()
    x.transpose_(0, 1)
    x.transpose_(1, 2)
    x = deprocess(x).clamp(0, 1)

    return Image.fromarray(np.uint8(x.cpu() * 255))

In [ ]:
def deep_dream(image, num_layer, iterations, step, octave_scale, num_octave):
    if num_octave > 0:
        new_image = image.filter(ImageFilter.GaussianBlur(2))

        if(new_image.size[0] / octave_scale < 1 or new_image.size[1] / octave_scale < 1):
            size = new_image.size
        else:
            size = (int(new_image.size[0] / octave_scale), int(new_image.size[1] / octave_scale))
        new_image = new_image.resize(size, Image.ANTIALIAS)

        new_image = deep_dream(new_image, num_layer, iterations, step, octave_scale, num_octave - 1)

        new_image = new_image.resize(image.size, Image.ANTIALIAS)
        image = ImageChops.blend(image, new_image, 0.6)

    ret = helper(image, num_layer, iterations, step)

    return ret.resize(image.size)

In [ ]:
image = Image.open('le_bain_turc.jpg')
deep_dream(image, NUM_LAYER, ITERATIONS, STEP, OCTAVE_SCALE, NUM_OCTAVE)